In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# ML
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score
import xgboost as xgb
import lightgbm as lgb
import joblib

# GPU check
import torch
GPU_AVAILABLE = torch.cuda.is_available()

# Paths
BASE_DIR = Path.cwd()
DATA_DIR = BASE_DIR / 'data'
MODEL_DIR = BASE_DIR / 'models' / 'signal_generator_v3'
MODEL_DIR.mkdir(parents=True, exist_ok=True)

print("="*60)
print("🚀 FOREX SIGNAL GENERATOR V3 - BUY ONLY")
print("="*60)
print(f"✓ Libraries loaded")
print(f"✓ GPU Available: {GPU_AVAILABLE}")
if GPU_AVAILABLE:
    print(f"  Device: {torch.cuda.get_device_name(0)}")
print(f"✓ Models: {MODEL_DIR}")

🚀 FOREX SIGNAL GENERATOR V3 - BUY ONLY
✓ Libraries loaded
✓ GPU Available: True
  Device: NVIDIA GeForce RTX 5060 Laptop GPU
✓ Models: c:\Users\Acer\Desktop\Forex-Signal-App\models\signal_generator_v3


## 1. Data Loading

In [2]:
# Load data (same as V2)
train_df = pd.read_csv(DATA_DIR / 'EUR_USD_1min.csv')
test_df = pd.read_csv(DATA_DIR / 'EUR_USD_test.csv')

print(f"Train data: {len(train_df):,} rows")
print(f"Test data: {len(test_df):,} rows")

Train data: 1,859,492 rows
Test data: 296,778 rows


## 2. Feature Engineering (Same as V2)

In [3]:
def add_technical_indicators(df):
    """Add comprehensive technical indicators optimized for BUY signals."""
    df = df.copy()
    
    # ==================== TREND INDICATORS ====================
    # Moving Averages
    for period in [5, 10, 20, 50, 100, 200]:
        df[f'sma_{period}'] = df['close'].rolling(period).mean()
        df[f'ema_{period}'] = df['close'].ewm(span=period, adjust=False).mean()
    
    # MA Crossovers - BUY signals
    df['sma_5_20_cross'] = (df['sma_5'] > df['sma_20']).astype(int)
    df['sma_20_50_cross'] = (df['sma_20'] > df['sma_50']).astype(int)
    df['ema_10_50_cross'] = (df['ema_10'] > df['ema_50']).astype(int)
    df['golden_cross'] = (df['sma_50'] > df['sma_200']).astype(int)
    
    # Price vs MAs
    df['price_vs_sma20'] = (df['close'] - df['sma_20']) / df['sma_20'] * 100
    df['price_vs_sma50'] = (df['close'] - df['sma_50']) / df['sma_50'] * 100
    df['price_vs_ema20'] = (df['close'] - df['ema_20']) / df['ema_20'] * 100
    df['price_above_all_ma'] = ((df['close'] > df['sma_20']) & 
                                 (df['close'] > df['sma_50']) & 
                                 (df['close'] > df['ema_20'])).astype(int)
    
    # ==================== MOMENTUM INDICATORS ====================
    # RSI
    for period in [7, 14, 21]:
        delta = df['close'].diff()
        gain = delta.where(delta > 0, 0).rolling(period).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(period).mean()
        rs = gain / (loss + 1e-10)
        df[f'rsi_{period}'] = 100 - (100 / (1 + rs))
    
    df['rsi_oversold'] = (df['rsi_14'] < 30).astype(int)
    df['rsi_bullish'] = ((df['rsi_14'] > 50) & (df['rsi_14'] < 70)).astype(int)
    
    # MACD
    ema12 = df['close'].ewm(span=12, adjust=False).mean()
    ema26 = df['close'].ewm(span=26, adjust=False).mean()
    df['macd'] = ema12 - ema26
    df['macd_signal'] = df['macd'].ewm(span=9, adjust=False).mean()
    df['macd_hist'] = df['macd'] - df['macd_signal']
    df['macd_cross'] = (df['macd'] > df['macd_signal']).astype(int)
    df['macd_bullish'] = ((df['macd'] > df['macd_signal']) & (df['macd_hist'] > 0)).astype(int)
    
    # Stochastic
    for period in [14, 21]:
        low_min = df['low'].rolling(period).min()
        high_max = df['high'].rolling(period).max()
        df[f'stoch_k_{period}'] = 100 * (df['close'] - low_min) / (high_max - low_min + 1e-10)
        df[f'stoch_d_{period}'] = df[f'stoch_k_{period}'].rolling(3).mean()
    
    df['stoch_oversold'] = (df['stoch_k_14'] < 20).astype(int)
    
    # ROC
    for period in [5, 10, 20]:
        df[f'roc_{period}'] = df['close'].pct_change(period) * 100
    
    # Momentum
    df['momentum_10'] = df['close'] - df['close'].shift(10)
    df['momentum_20'] = df['close'] - df['close'].shift(20)
    
    # ==================== VOLATILITY INDICATORS ====================
    high_low = df['high'] - df['low']
    high_close = abs(df['high'] - df['close'].shift())
    low_close = abs(df['low'] - df['close'].shift())
    tr = pd.concat([high_low, high_close, low_close], axis=1).max(axis=1)
    df['atr_14'] = tr.rolling(14).mean()
    df['atr_20'] = tr.rolling(20).mean()
    df['atr_pips'] = df['atr_14'] * 10000
    df['atr_pct'] = df['atr_14'] / df['close'] * 100
    
    # Bollinger Bands
    df['bb_middle'] = df['close'].rolling(20).mean()
    bb_std = df['close'].rolling(20).std()
    df['bb_upper'] = df['bb_middle'] + 2 * bb_std
    df['bb_lower'] = df['bb_middle'] - 2 * bb_std
    df['bb_width'] = (df['bb_upper'] - df['bb_lower']) / df['bb_middle'] * 100
    df['bb_position'] = (df['close'] - df['bb_lower']) / (df['bb_upper'] - df['bb_lower'] + 1e-10)
    df['bb_squeeze'] = (df['bb_width'] < df['bb_width'].rolling(50).mean()).astype(int)
    
    # ==================== CANDLE PATTERNS ====================
    df['candle_body'] = df['close'] - df['open']
    df['candle_range'] = df['high'] - df['low']
    df['candle_body_pct'] = df['candle_body'] / (df['candle_range'] + 1e-10)
    df['upper_shadow'] = df['high'] - df[['open', 'close']].max(axis=1)
    df['lower_shadow'] = df[['open', 'close']].min(axis=1) - df['low']
    
    df['is_bullish'] = (df['close'] > df['open']).astype(int)
    df['is_hammer'] = ((df['lower_shadow'] > df['candle_body'].abs() * 2) & 
                       (df['upper_shadow'] < df['candle_body'].abs() * 0.5)).astype(int)
    df['bullish_engulfing'] = ((df['is_bullish'] == 1) & 
                               (df['is_bullish'].shift(1) == 0) &
                               (df['close'] > df['open'].shift(1))).astype(int)
    
    # ==================== SUPPORT/RESISTANCE ====================
    df['pivot'] = (df['high'].shift() + df['low'].shift() + df['close'].shift()) / 3
    df['r1'] = 2 * df['pivot'] - df['low'].shift()
    df['s1'] = 2 * df['pivot'] - df['high'].shift()
    df['r2'] = df['pivot'] + (df['high'].shift() - df['low'].shift())
    df['s2'] = df['pivot'] - (df['high'].shift() - df['low'].shift())
    df['near_support'] = (df['close'] < df['s1'] * 1.001).astype(int)
    
    # ==================== TREND STRENGTH ====================
    df['trend_short'] = np.where(df['ema_10'] > df['ema_20'], 1, -1)
    df['trend_medium'] = np.where(df['ema_20'] > df['ema_50'], 1, -1)
    df['trend_long'] = np.where(df['ema_50'] > df['ema_200'], 1, -1)
    df['trend_alignment'] = df['trend_short'] + df['trend_medium'] + df['trend_long']
    df['strong_uptrend'] = (df['trend_alignment'] == 3).astype(int)
    
    # ==================== BUY SCORE ====================
    df['buy_score'] = (
        df['macd_bullish'] +
        df['rsi_bullish'] +
        df['sma_5_20_cross'] +
        df['golden_cross'] +
        df['price_above_all_ma'] +
        df['strong_uptrend']
    )
    
    return df

# Apply
print("Adding technical indicators...")
train_df = add_technical_indicators(train_df)
test_df = add_technical_indicators(test_df)

print(f"✓ Features: {len(train_df.columns)} columns")

Adding technical indicators...
✓ Features: 76 columns
✓ Features: 76 columns


## 3. V3 Label: BUY vs NOT_BUY (Binary Classification)

**V2-аас ялгаатай нь:**
- V2: BUY(1) vs SELL(-1) → дараа нь SELL хасах
- V3: BUY(1) vs NOT_BUY(0) → шууд binary classification

In [4]:
def create_buy_only_labels(df, forward_periods=60, tp_pips=15, sl_pips=10):
    """
    V3: BUY-only labels.
    
    BUY (1): Price goes up by tp_pips within forward_periods WITHOUT hitting sl_pips first
    NOT_BUY (0): Either SL hit first, or neither TP nor SL hit
    
    Risk:Reward = 1:1.5 (10 SL : 15 TP)
    """
    df = df.copy()
    tp_move = tp_pips * 0.0001
    sl_move = sl_pips * 0.0001
    
    print(f"Creating BUY-only labels...")
    print(f"  TP: {tp_pips} pips, SL: {sl_pips} pips (R:R = 1:{tp_pips/sl_pips:.1f})")
    print(f"  Forward period: {forward_periods} bars")
    
    labels = []
    
    for i in range(len(df)):
        if i + forward_periods >= len(df):
            labels.append(0)  # Not enough data
            continue
        
        entry = df['close'].iloc[i]
        tp_price = entry + tp_move
        sl_price = entry - sl_move
        
        # Check future bars
        is_buy = 0
        for j in range(1, forward_periods + 1):
            future_high = df['high'].iloc[i + j]
            future_low = df['low'].iloc[i + j]
            
            # Check if SL hit first
            if future_low <= sl_price:
                is_buy = 0
                break
            
            # Check if TP hit
            if future_high >= tp_price:
                is_buy = 1
                break
        
        labels.append(is_buy)
    
    df['is_buy'] = labels
    return df

# Faster vectorized version
def create_buy_only_labels_fast(df, forward_periods=60, tp_pips=15, sl_pips=10):
    """
    V3: BUY-only labels (vectorized, faster).
    Simplified: TP reachable AND SL not hit within forward_periods.
    """
    df = df.copy()
    tp_move = tp_pips * 0.0001
    sl_move = sl_pips * 0.0001
    
    print(f"Creating BUY-only labels (V3)...")
    print(f"  TP: {tp_pips} pips, SL: {sl_pips} pips (R:R = 1:{tp_pips/sl_pips:.1f})")
    
    # Calculate TP and SL prices
    df['tp_price'] = df['close'] + tp_move
    df['sl_price'] = df['close'] - sl_move
    
    # Future max/min within forward_periods
    df['future_max'] = df['high'].rolling(window=forward_periods).max().shift(-forward_periods)
    df['future_min'] = df['low'].rolling(window=forward_periods).min().shift(-forward_periods)
    
    # BUY = TP reachable AND SL not hit
    df['tp_reachable'] = df['future_max'] >= df['tp_price']
    df['sl_hit'] = df['future_min'] <= df['sl_price']
    
    # Conservative: BUY only if TP hit and SL not hit
    df['is_buy'] = ((df['tp_reachable']) & (~df['sl_hit'])).astype(int)
    
    # Cleanup
    df.drop(['tp_price', 'sl_price', 'future_max', 'future_min', 'tp_reachable', 'sl_hit'], axis=1, inplace=True)
    
    return df

# Create labels
train_df = create_buy_only_labels_fast(train_df, forward_periods=60, tp_pips=15, sl_pips=10)
test_df = create_buy_only_labels_fast(test_df, forward_periods=60, tp_pips=15, sl_pips=10)

# Label distribution
print(f"\n📊 Label Distribution (Train):")
buy_count = train_df['is_buy'].sum()
total = len(train_df)
print(f"  BUY:     {buy_count:,} ({buy_count/total*100:.1f}%)")
print(f"  NOT_BUY: {total - buy_count:,} ({(total-buy_count)/total*100:.1f}%)")

print(f"\n📊 Label Distribution (Test):")
buy_count_test = test_df['is_buy'].sum()
total_test = len(test_df)
print(f"  BUY:     {buy_count_test:,} ({buy_count_test/total_test*100:.1f}%)")
print(f"  NOT_BUY: {total_test - buy_count_test:,} ({(total_test-buy_count_test)/total_test*100:.1f}%)")

Creating BUY-only labels (V3)...
  TP: 15 pips, SL: 10 pips (R:R = 1:1.5)
Creating BUY-only labels (V3)...
  TP: 15 pips, SL: 10 pips (R:R = 1:1.5)

📊 Label Distribution (Train):
  BUY:     179,895 (9.7%)
  NOT_BUY: 1,679,597 (90.3%)

📊 Label Distribution (Test):
  BUY:     38,497 (13.0%)
  NOT_BUY: 258,281 (87.0%)
Creating BUY-only labels (V3)...
  TP: 15 pips, SL: 10 pips (R:R = 1:1.5)

📊 Label Distribution (Train):
  BUY:     179,895 (9.7%)
  NOT_BUY: 1,679,597 (90.3%)

📊 Label Distribution (Test):
  BUY:     38,497 (13.0%)
  NOT_BUY: 258,281 (87.0%)


## 4. Prepare Training Data

In [5]:
# Select features
exclude_cols = ['timestamp', 'time', 'date', 'is_buy', 'move_strength', 
                'actual_profit_pips', 'actual_drawdown_pips',
                'open', 'high', 'low', 'close', 'volume', 'tick_volume']

feature_cols = [col for col in train_df.columns if col not in exclude_cols]
print(f"Features: {len(feature_cols)}")

# Drop NaN rows
train_clean = train_df.dropna(subset=feature_cols + ['is_buy']).copy()
test_clean = test_df.dropna(subset=feature_cols + ['is_buy']).copy()

print(f"\nTrain samples: {len(train_clean):,}")
print(f"Test samples: {len(test_clean):,}")

# Prepare X, y
X_train = train_clean[feature_cols].values
y_train = train_clean['is_buy'].values

X_test = test_clean[feature_cols].values
y_test = test_clean['is_buy'].values

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"\n✓ Data ready")
print(f"  X_train: {X_train_scaled.shape}")
print(f"  X_test: {X_test_scaled.shape}")
print(f"  BUY ratio (train): {y_train.mean()*100:.1f}%")
print(f"  BUY ratio (test): {y_test.mean()*100:.1f}%")

Features: 70

Train samples: 1,859,293
Test samples: 296,579

Train samples: 1,859,293
Test samples: 296,579

✓ Data ready
  X_train: (1859293, 70)
  X_test: (296579, 70)
  BUY ratio (train): 9.7%
  BUY ratio (test): 13.0%

✓ Data ready
  X_train: (1859293, 70)
  X_test: (296579, 70)
  BUY ratio (train): 9.7%
  BUY ratio (test): 13.0%


## 5. Train V3 Models (BUY vs NOT_BUY)

In [6]:
print("="*60)
print("🎯 V3: TRAINING BUY vs NOT_BUY MODELS")
print("="*60)

# Calculate class weight for imbalanced data
neg_count = (y_train == 0).sum()
pos_count = (y_train == 1).sum()
scale_pos_weight = neg_count / pos_count
print(f"\nClass imbalance: {neg_count:,} NOT_BUY vs {pos_count:,} BUY")
print(f"Scale pos weight: {scale_pos_weight:.2f}")

# ==================== Model 1: XGBoost ====================
print("\nTraining XGBoost...")
xgb_model = xgb.XGBClassifier(
    n_estimators=500,
    max_depth=8,
    learning_rate=0.03,
    subsample=0.8,
    colsample_bytree=0.8,
    scale_pos_weight=scale_pos_weight,
    random_state=42,
    eval_metric='logloss',
    verbosity=0,
    tree_method='hist',
    device='cuda' if GPU_AVAILABLE else 'cpu'
)
xgb_model.fit(X_train_scaled, y_train)
xgb_pred = xgb_model.predict(X_test_scaled)
xgb_proba = xgb_model.predict_proba(X_test_scaled)
xgb_acc = accuracy_score(y_test, xgb_pred)
xgb_prec = precision_score(y_test, xgb_pred, zero_division=0)
xgb_rec = recall_score(y_test, xgb_pred, zero_division=0)
xgb_f1 = f1_score(y_test, xgb_pred, zero_division=0)
print(f"  ✓ XGBoost - Acc: {xgb_acc*100:.2f}%, Prec: {xgb_prec*100:.2f}%, Rec: {xgb_rec*100:.2f}%, F1: {xgb_f1:.3f}")

# ==================== Model 2: LightGBM ====================
print("\nTraining LightGBM...")
lgb_model = lgb.LGBMClassifier(
    n_estimators=500,
    max_depth=8,
    learning_rate=0.03,
    subsample=0.8,
    colsample_bytree=0.8,
    scale_pos_weight=scale_pos_weight,
    random_state=42,
    verbose=-1,
    n_jobs=-1
)
lgb_model.fit(X_train_scaled, y_train)
lgb_pred = lgb_model.predict(X_test_scaled)
lgb_proba = lgb_model.predict_proba(X_test_scaled)
lgb_acc = accuracy_score(y_test, lgb_pred)
lgb_prec = precision_score(y_test, lgb_pred, zero_division=0)
lgb_rec = recall_score(y_test, lgb_pred, zero_division=0)
lgb_f1 = f1_score(y_test, lgb_pred, zero_division=0)
print(f"  ✓ LightGBM - Acc: {lgb_acc*100:.2f}%, Prec: {lgb_prec*100:.2f}%, Rec: {lgb_rec*100:.2f}%, F1: {lgb_f1:.3f}")

# ==================== Model 3: Random Forest ====================
print("\nTraining Random Forest...")
rf_model = RandomForestClassifier(
    n_estimators=300,
    max_depth=15,
    min_samples_split=10,
    class_weight='balanced',
    random_state=42,
    n_jobs=-1
)
rf_model.fit(X_train_scaled, y_train)
rf_pred = rf_model.predict(X_test_scaled)
rf_proba = rf_model.predict_proba(X_test_scaled)
rf_acc = accuracy_score(y_test, rf_pred)
rf_prec = precision_score(y_test, rf_pred, zero_division=0)
rf_rec = recall_score(y_test, rf_pred, zero_division=0)
rf_f1 = f1_score(y_test, rf_pred, zero_division=0)
print(f"  ✓ Random Forest - Acc: {rf_acc*100:.2f}%, Prec: {rf_prec*100:.2f}%, Rec: {rf_rec*100:.2f}%, F1: {rf_f1:.3f}")

# Summary
print(f"\n{'='*60}")
print(f"📊 V3 MODEL COMPARISON (BUY vs NOT_BUY):")
print(f"   XGBoost:       {xgb_acc*100:.2f}% acc, {xgb_prec*100:.2f}% precision, {xgb_f1:.3f} F1")
print(f"   LightGBM:      {lgb_acc*100:.2f}% acc, {lgb_prec*100:.2f}% precision, {lgb_f1:.3f} F1")
print(f"   Random Forest: {rf_acc*100:.2f}% acc, {rf_prec*100:.2f}% precision, {rf_f1:.3f} F1")

🎯 V3: TRAINING BUY vs NOT_BUY MODELS

Class imbalance: 1,679,398 NOT_BUY vs 179,895 BUY
Scale pos weight: 9.34

Training XGBoost...
  ✓ XGBoost - Acc: 58.28%, Prec: 20.63%, Rec: 77.76%, F1: 0.326

Training LightGBM...
  ✓ XGBoost - Acc: 58.28%, Prec: 20.63%, Rec: 77.76%, F1: 0.326

Training LightGBM...
  ✓ LightGBM - Acc: 56.28%, Prec: 20.25%, Rec: 80.63%, F1: 0.324

Training Random Forest...
  ✓ LightGBM - Acc: 56.28%, Prec: 20.25%, Rec: 80.63%, F1: 0.324

Training Random Forest...
  ✓ Random Forest - Acc: 61.49%, Prec: 21.61%, Rec: 74.84%, F1: 0.335

📊 V3 MODEL COMPARISON (BUY vs NOT_BUY):
   XGBoost:       58.28% acc, 20.63% precision, 0.326 F1
   LightGBM:      56.28% acc, 20.25% precision, 0.324 F1
   Random Forest: 61.49% acc, 21.61% precision, 0.335 F1
  ✓ Random Forest - Acc: 61.49%, Prec: 21.61%, Rec: 74.84%, F1: 0.335

📊 V3 MODEL COMPARISON (BUY vs NOT_BUY):
   XGBoost:       58.28% acc, 20.63% precision, 0.326 F1
   LightGBM:      56.28% acc, 20.25% precision, 0.324 F1
   Ra

## 6. V3 Ensemble + Confidence Analysis

In [7]:
# ==================== ENSEMBLE ====================
print("="*60)
print("🔍 V3 ENSEMBLE + CONFIDENCE ANALYSIS")
print("="*60)

# Ensemble probabilities (equal weight)
avg_proba = (xgb_proba + lgb_proba + rf_proba) / 3

# BUY probability
buy_prob = avg_proba[:, 1] * 100  # As percentage

# Model agreement
all_agree_buy = (xgb_pred == 1) & (lgb_pred == 1) & (rf_pred == 1)
print(f"\nAll models agree on BUY: {all_agree_buy.sum():,} times")

# Confidence = BUY probability (since it's binary BUY vs NOT_BUY)
confidence = buy_prob.copy()

# Boost confidence when all agree
confidence[all_agree_buy] = np.minimum(confidence[all_agree_buy] + 5, 100)

# ==================== ACCURACY BY CONFIDENCE ====================
print(f"\n📊 BUY Signal Accuracy by Confidence Level:")
print("-"*60)

results = []
for min_conf in [50, 60, 70, 75, 80, 85, 90, 95]:
    mask = confidence >= min_conf
    if mask.sum() > 0:
        # Among high confidence predictions, how many are actually BUY?
        actual_buy = y_test[mask].sum()
        total_pred = mask.sum()
        precision = actual_buy / total_pred if total_pred > 0 else 0
        
        results.append((min_conf, precision, total_pred, actual_buy))
        print(f"  Conf >= {min_conf}%: {precision*100:.1f}% precision ({total_pred:,} signals, {actual_buy:,} correct)")

# Best threshold
print(f"\n🎯 Optimal thresholds:")
for min_conf, prec, total, correct in results:
    if prec >= 0.70 and total >= 20:
        print(f"   ⭐ {min_conf}%+: {prec*100:.1f}% precision, {total} signals")

🔍 V3 ENSEMBLE + CONFIDENCE ANALYSIS

All models agree on BUY: 128,844 times

📊 BUY Signal Accuracy by Confidence Level:
------------------------------------------------------------
  Conf >= 50%: 20.9% precision (143,501 signals, 30,023 correct)
  Conf >= 60%: 22.5% precision (122,127 signals, 27,485 correct)
  Conf >= 70%: 26.5% precision (76,045 signals, 20,166 correct)
  Conf >= 75%: 28.4% precision (53,635 signals, 15,233 correct)
  Conf >= 80%: 30.2% precision (28,849 signals, 8,725 correct)
  Conf >= 85%: 35.1% precision (2,804 signals, 984 correct)

🎯 Optimal thresholds:


## 7. Backtest V3

In [8]:
def backtest_v3(df, confidence, y_actual, min_confidence=80, forward_periods=60):
    """
    Backtest V3 BUY signals with dynamic SL/TP.
    """
    results = []
    
    # High confidence BUY signals
    buy_mask = confidence >= min_confidence
    signal_indices = np.where(buy_mask)[0]
    
    print(f"Testing {len(signal_indices)} BUY signals (conf >= {min_confidence}%)")
    
    for idx in signal_indices:
        if idx + forward_periods >= len(df):
            continue
        
        entry_price = df['close'].iloc[idx]
        conf = confidence[idx]
        
        # Dynamic SL/TP based on ATR
        atr = df['atr_14'].iloc[idx] if 'atr_14' in df.columns else entry_price * 0.0008
        atr_pips = atr * 10000
        
        sl_pips = max(10, min(20, atr_pips * 1.5))
        tp_pips = max(15, min(30, atr_pips * 2.0))
        if tp_pips < sl_pips * 1.5:
            tp_pips = sl_pips * 1.5
        
        sl_price = entry_price - sl_pips * 0.0001
        tp_price = entry_price + tp_pips * 0.0001
        
        # Future prices
        future_slice = df.iloc[idx+1:idx+forward_periods+1]
        
        # Check bar by bar for more accurate result
        pnl_pips = 0
        result = 'TIMEOUT'
        for _, bar in future_slice.iterrows():
            if bar['low'] <= sl_price:
                pnl_pips = -sl_pips
                result = 'LOSS'
                break
            if bar['high'] >= tp_price:
                pnl_pips = tp_pips
                result = 'WIN'
                break
        
        if result == 'TIMEOUT':
            final_price = df['close'].iloc[idx+forward_periods]
            pnl_pips = (final_price - entry_price) * 10000
            result = 'WIN' if pnl_pips > 0 else 'LOSS'
        
        results.append({
            'confidence': conf,
            'entry_price': entry_price,
            'sl_pips': sl_pips,
            'tp_pips': tp_pips,
            'pnl_pips': pnl_pips,
            'result': result,
            'actual_buy': y_actual[idx]
        })
    
    return pd.DataFrame(results)

# Run backtest
print("="*70)
print("📊 V3 BACKTEST - BUY ONLY WITH DYNAMIC SL/TP")
print("="*70)

v3_results = {}
for min_conf in [70, 75, 80, 85, 90]:
    bt_results = backtest_v3(
        test_clean.reset_index(drop=True),
        confidence,
        y_test,
        min_confidence=min_conf,
        forward_periods=60
    )
    
    if len(bt_results) > 0:
        wins = (bt_results['result'] == 'WIN').sum()
        total = len(bt_results)
        win_rate = wins / total * 100
        total_pips = bt_results['pnl_pips'].sum()
        avg_pips = bt_results['pnl_pips'].mean()
        
        # Profit factor
        gross_profit = bt_results[bt_results['pnl_pips'] > 0]['pnl_pips'].sum()
        gross_loss = abs(bt_results[bt_results['pnl_pips'] < 0]['pnl_pips'].sum())
        pf = gross_profit / gross_loss if gross_loss > 0 else float('inf')
        
        v3_results[min_conf] = {
            'signals': total,
            'win_rate': win_rate,
            'total_pips': total_pips,
            'avg_pips': avg_pips,
            'pf': pf
        }
        
        emoji = "✅" if total_pips > 0 else "❌"
        print(f"\nConf >= {min_conf}%: {total} signals")
        print(f"   Win Rate: {win_rate:.1f}% | Total: {total_pips:+,.1f} pips {emoji}")
        print(f"   Avg: {avg_pips:+.2f} pips | PF: {pf:.2f}")

📊 V3 BACKTEST - BUY ONLY WITH DYNAMIC SL/TP
Testing 76045 BUY signals (conf >= 70%)
Testing 76045 BUY signals (conf >= 70%)

Conf >= 70%: 76035 signals
   Win Rate: 45.3% | Total: +29,596.3 pips ✅
   Avg: +0.39 pips | PF: 1.08
Testing 53635 BUY signals (conf >= 75%)

Conf >= 70%: 76035 signals
   Win Rate: 45.3% | Total: +29,596.3 pips ✅
   Avg: +0.39 pips | PF: 1.08
Testing 53635 BUY signals (conf >= 75%)

Conf >= 75%: 53629 signals
   Win Rate: 45.2% | Total: +27,089.2 pips ✅
   Avg: +0.51 pips | PF: 1.10
Testing 28849 BUY signals (conf >= 80%)

Conf >= 75%: 53629 signals
   Win Rate: 45.2% | Total: +27,089.2 pips ✅
   Avg: +0.51 pips | PF: 1.10
Testing 28849 BUY signals (conf >= 80%)

Conf >= 80%: 28849 signals
   Win Rate: 44.5% | Total: +14,421.5 pips ✅
   Avg: +0.50 pips | PF: 1.10
Testing 2804 BUY signals (conf >= 85%)

Conf >= 80%: 28849 signals
   Win Rate: 44.5% | Total: +14,421.5 pips ✅
   Avg: +0.50 pips | PF: 1.10
Testing 2804 BUY signals (conf >= 85%)

Conf >= 85%: 2804 s

## 8. 📊 V2 vs V3 Comparison

In [9]:
# V2 results (from previous notebook)
v2_results = {
    75: {'signals': 279, 'win_rate': 48.4, 'total_pips': 937.0, 'pf': 1.76},
    80: {'signals': 105, 'win_rate': 61.9, 'total_pips': 671.1, 'pf': 3.10},
    85: {'signals': 48, 'win_rate': 68.8, 'total_pips': 387.2, 'pf': 4.82},
    90: {'signals': 9, 'win_rate': 100.0, 'total_pips': 119.6, 'pf': float('inf')}
}

print("="*80)
print("📊 V2 vs V3 COMPARISON")
print("="*80)
print(f"\n{'Conf':<8} | {'V2 Signals':<12} | {'V3 Signals':<12} | {'V2 WR':<10} | {'V3 WR':<10} | {'V2 Pips':<12} | {'V3 Pips':<12} | {'V2 PF':<8} | {'V3 PF':<8}")
print("-"*110)

for conf in [75, 80, 85, 90]:
    v2 = v2_results.get(conf, {})
    v3 = v3_results.get(conf, {})
    
    v2_sig = v2.get('signals', 0)
    v3_sig = v3.get('signals', 0)
    v2_wr = v2.get('win_rate', 0)
    v3_wr = v3.get('win_rate', 0)
    v2_pips = v2.get('total_pips', 0)
    v3_pips = v3.get('total_pips', 0)
    v2_pf = v2.get('pf', 0)
    v3_pf = v3.get('pf', 0)
    
    # Better indicator
    wr_better = "V3 ✓" if v3_wr > v2_wr else "V2 ✓" if v2_wr > v3_wr else "="
    pips_better = "V3 ✓" if v3_pips > v2_pips else "V2 ✓" if v2_pips > v3_pips else "="
    
    print(f"{conf}%+{'':<4} | {v2_sig:<12} | {v3_sig:<12} | {v2_wr:<10.1f} | {v3_wr:<10.1f} | {v2_pips:<+12.1f} | {v3_pips:<+12.1f} | {v2_pf:<8.2f} | {v3_pf:<8.2f}")

print("\n" + "="*80)
print("📝 ANALYSIS:")
print("="*80)
print("""
V2: BUY vs SELL classification → дараа нь зөвхөн BUY сигнал ашиглах
V3: BUY vs NOT_BUY binary classification → шууд BUY таамаглах

Давуу тал:
- V3 нь илүү цэвэр арга (зөвхөн нэг зорилготой)
- V3 нь SELL сигналын нөлөөнөөс чөлөөтэй
""")

📊 V2 vs V3 COMPARISON

Conf     | V2 Signals   | V3 Signals   | V2 WR      | V3 WR      | V2 Pips      | V3 Pips      | V2 PF    | V3 PF   
--------------------------------------------------------------------------------------------------------------
75%+     | 279          | 53629        | 48.4       | 45.2       | +937.0       | +27089.2     | 1.76     | 1.10    
80%+     | 105          | 28849        | 61.9       | 44.5       | +671.1       | +14421.5     | 3.10     | 1.10    
85%+     | 48           | 2804         | 68.8       | 47.3       | +387.2       | +4030.7      | 4.82     | 1.30    
90%+     | 9            | 0            | 100.0      | 0.0        | +119.6       | +0.0         | inf      | 0.00    

📝 ANALYSIS:

V2: BUY vs SELL classification → дараа нь зөвхөн BUY сигнал ашиглах
V3: BUY vs NOT_BUY binary classification → шууд BUY таамаглах

Давуу тал:
- V3 нь илүү цэвэр арга (зөвхөн нэг зорилготой)
- V3 нь SELL сигналын нөлөөнөөс чөлөөтэй



## 9. Save V3 Models

In [10]:
# Save V3 models
print("Saving V3 models...")

joblib.dump(xgb_model, MODEL_DIR / 'xgboost_v3.joblib')
joblib.dump(lgb_model, MODEL_DIR / 'lightgbm_v3.joblib')
joblib.dump(rf_model, MODEL_DIR / 'rf_v3.joblib')
joblib.dump(scaler, MODEL_DIR / 'scaler_v3.joblib')
joblib.dump(feature_cols, MODEL_DIR / 'feature_cols_v3.joblib')

# Save config
config_v3 = {
    'version': 'v3',
    'mode': 'BUY_ONLY_BINARY',
    'label': 'BUY vs NOT_BUY',
    'tp_pips': 15,
    'sl_pips': 10,
    'forward_periods': 60,
    'features_count': len(feature_cols)
}
joblib.dump(config_v3, MODEL_DIR / 'config_v3.joblib')

print(f"\n✅ V3 Models saved to {MODEL_DIR}")
print(f"\nFiles:")
for f in sorted(MODEL_DIR.glob('*_v3.joblib')):
    print(f"   {f.name}")

Saving V3 models...

✅ V3 Models saved to c:\Users\Acer\Desktop\Forex-Signal-App\models\signal_generator_v3

Files:
   config_v3.joblib
   feature_cols_v3.joblib
   lightgbm_v3.joblib
   rf_v3.joblib
   scaler_v3.joblib
   xgboost_v3.joblib

✅ V3 Models saved to c:\Users\Acer\Desktop\Forex-Signal-App\models\signal_generator_v3

Files:
   config_v3.joblib
   feature_cols_v3.joblib
   lightgbm_v3.joblib
   rf_v3.joblib
   scaler_v3.joblib
   xgboost_v3.joblib


## 10. 📊 Final Summary

In [11]:
print("="*70)
print("📊 FOREX SIGNAL GENERATOR V3 - FINAL SUMMARY")
print("="*70)

# Get best V3 result
best_conf = 80
if best_conf in v3_results:
    v3_best = v3_results[best_conf]
    v2_best = v2_results[best_conf]
    
    print(f"""
🎯 V3 APPROACH: BUY vs NOT_BUY Binary Classification

📈 V3 Results (80%+ confidence):
   Signals:    {v3_best['signals']}
   Win Rate:   {v3_best['win_rate']:.1f}%
   Total Pips: {v3_best['total_pips']:+.1f}
   Profit Factor: {v3_best['pf']:.2f}

📊 V2 Results (80%+ confidence) for comparison:
   Signals:    {v2_best['signals']}
   Win Rate:   {v2_best['win_rate']:.1f}%
   Total Pips: {v2_best['total_pips']:+.1f}
   Profit Factor: {v2_best['pf']:.2f}

✅ V3 MODEL SAVED: models/signal_generator_v3/
""")
else:
    print("\nNo results for 80%+ confidence. Check the backtest results above.")

print("="*70)
print("🚀 V3 READY!")
print("="*70)

📊 FOREX SIGNAL GENERATOR V3 - FINAL SUMMARY

🎯 V3 APPROACH: BUY vs NOT_BUY Binary Classification

📈 V3 Results (80%+ confidence):
   Signals:    28849
   Win Rate:   44.5%
   Total Pips: +14421.5
   Profit Factor: 1.10

📊 V2 Results (80%+ confidence) for comparison:
   Signals:    105
   Win Rate:   61.9%
   Total Pips: +671.1
   Profit Factor: 3.10

✅ V3 MODEL SAVED: models/signal_generator_v3/

🚀 V3 READY!
